In [1]:
import openpyxl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
import datetime
import math
import pandas as pd
from scipy import stats
import seaborn as sn

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Drops specific values in a dataframe
# if flag = 1, drops values less than 'val'
# if flag = 0, drops values greater than 'val'
def drop_values(dataframe, column, val, flag):
    if flag:
        return dataframe.drop(dataframe[column < val].index)
    else:
        return dataframe.drop(dataframe[column > val].index)

In [7]:
egg_1_dataframe_1 = pd.read_csv('RAW_DATA/Egg#1/2020-05-16 Download_egg0080446e6e980130.csv')
egg_1_dataframe_2 = pd.read_csv('RAW_DATA/Egg#1/2020-05-25 Download_egg0080446e6e980130.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_1_dataframe_1 = egg_1_dataframe_1.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_1_dataframe_2 = egg_1_dataframe_2.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_1_dataframe_1['Timestamp'] = pd.to_datetime(egg_1_dataframe_1['Timestamp'])
egg_1_dataframe_2['Timestamp'] = pd.to_datetime(egg_1_dataframe_2['Timestamp'])
# Merge all the files into 1 dataframe
egg_1_dataframe = pd.concat((egg_1_dataframe_1, egg_1_dataframe_2))
egg_1_dataframe['Timestamp'] = pd.to_datetime(egg_1_dataframe['Timestamp'])
egg_1_dataframe = egg_1_dataframe.resample('60min', on='Timestamp').mean()
egg_1_dataframe.to_pickle('RAW_DATA_HOURLY/Egg#1_25May2020_hourly.pkl')

egg_2_dataframe_1 = pd.read_csv('RAW_DATA/Egg#2/2020-05-16 Download_egg0080446fa82b0130.csv')
egg_2_dataframe_2 = pd.read_csv('RAW_DATA/Egg#2/2020-05-25 Download_egg0080446fa82b0130.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_2_dataframe_1 = egg_2_dataframe_1.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_2_dataframe_2 = egg_2_dataframe_2.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_2_dataframe_1['Timestamp'] = pd.to_datetime(egg_2_dataframe_1['Timestamp'])
egg_2_dataframe_2['Timestamp'] = pd.to_datetime(egg_2_dataframe_2['Timestamp'])
# Merge all the files into 1 dataframe
egg_2_dataframe = pd.concat((egg_2_dataframe_1, egg_2_dataframe_2))
egg_2_dataframe['Timestamp'] = pd.to_datetime(egg_2_dataframe['Timestamp'])
egg_2_dataframe = egg_2_dataframe.resample('60min', on='Timestamp').mean()
egg_2_dataframe.to_pickle('RAW_DATA_HOURLY/Egg#2_25May2020_hourly.pkl')

egg_4_dataframe_1 = pd.read_csv('RAW_DATA/Egg#4/2020-05-16 Download_egg00804426a6980132.csv')
egg_4_dataframe_2 = pd.read_csv('RAW_DATA/Egg#4/2020-05-25 Download_egg00804426a6980132.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_4_dataframe_1 = egg_4_dataframe_1.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_4_dataframe_2 = egg_4_dataframe_2.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_4_dataframe_1['Timestamp'] = pd.to_datetime(egg_4_dataframe_1['Timestamp'])
egg_4_dataframe_2['Timestamp'] = pd.to_datetime(egg_4_dataframe_2['Timestamp'])
# Merge all the files into 1 dataframe
egg_4_dataframe = pd.concat((egg_4_dataframe_1, egg_4_dataframe_2))
egg_4_dataframe['Timestamp'] = pd.to_datetime(egg_4_dataframe['Timestamp'])
egg_4_dataframe = egg_4_dataframe.resample('60min', on='Timestamp').mean()
egg_4_dataframe.to_pickle('RAW_DATA_HOURLY/Egg#4_25May2020_hourly.pkl')

In [21]:
airpointer = pd.read_pickle('RAW_DATA_HOURLY/airpointer_25May2020_hourly.pkl')
airpointer

NO       NO2       NOx         O3  PM10_conc  \
Timestamp                                                                 
2020-05-06 13:00:00  1.405500  3.092333  4.498500  39.439667  14.807833   
2020-05-06 14:00:00  1.235167  2.711000  3.915167  40.489167  12.815500   
2020-05-06 15:00:00  1.483333  3.901500  5.595000  40.472333  15.562333   
2020-05-06 16:00:00  1.919000  5.137333  6.889833  40.594000  19.666000   
2020-05-06 17:00:00  2.494167  7.393667  9.502500  39.201833  21.422167   
...                       ...       ...       ...        ...        ...   
2020-05-25 11:00:00  0.365917  2.609533  2.975500  55.903200  19.268467   
2020-05-25 12:00:00  0.603183  6.032083  6.635250  56.402150  25.994967   
2020-05-25 13:00:00  0.628783  5.811367  6.428683  63.794417  30.028817   
2020-05-25 14:00:00  0.295300  3.735817  4.031100  65.850000  25.102767   
2020-05-25 15:00:00  0.455300  3.539067  3.994417  56.406867  23.447217   

                     PM10-2.5_conc  PM2.5_conc  Air_Temperature  \
Timestamp                                                         
2020-05-06 13:00:00      10.762833    4.045167        12.164500   
2020-05-06 14:00:00       8.869667    3.944333        12.281333   
2020-05-06 15:00:00      11.270000    4.290667        12.369500   
2020-05-06 16:00:00      15.035500    4.626500        12.851333   
2020-05-06 17:00:00      16.256500    5.165667        12.170000   
...                            ...         ...              ...   
2020-05-25 11:00:00      12.141783    7.127267       996.547950   
2020-05-25 12:00:00      14.871600   11.122967       996.351517   
2020-05-25 13:00:00      18.580050   11.447000       996.178783   
2020-05-25 14:00:00      14.554183   10.543317       996.079167   
2020-05-25 15:00:00      13.474400    9.971433       995.806150   

                     Relative_Humidity  
Timestamp                               
2020-05-06 13:00:00          29.069333  
2020-05-06 14:00:00          28.194667  
2020-05-06 15:00:00          27.624167  
2020-05-06 16:00:00          26.497000  
2020-05-06 17:00:00          27.646167  
...                                ...  
2020-05-25 11:00:00          30.645433  
2020-05-25 12:00:00          28.949983  
2020-05-25 13:00:00          28.876100  
2020-05-25 14:00:00          29.792433  
2020-05-25 15:00:00          29.564867  

[459 rows x 9 columns]

In [22]:
egg_1 = pd.read_pickle('RAW_DATA_HOURLY/Egg#1_25May2020_hourly.pkl')
egg_2 = pd.read_pickle('RAW_DATA_HOURLY/Egg#2_25May2020_hourly.pkl')
egg_4 = pd.read_pickle('RAW_DATA_HOURLY/Egg#4_25May2020_hourly.pkl')
airpointer = pd.read_pickle('RAW_DATA_HOURLY/airpointer_25May2020_hourly.pkl')

In [25]:
egg_1 = pd.read_pickle('RAW_DATA_HOURLY/Egg#1_25May2020_hourly.pkl')
egg_2 = pd.read_pickle('RAW_DATA_HOURLY/Egg#2_25May2020_hourly.pkl')
egg_4 = pd.read_pickle('RAW_DATA_HOURLY/Egg#4_25May2020_hourly.pkl')
airpointer = pd.read_pickle('RAW_DATA_HOURLY/airpointer_25May2020_hourly.pkl')

# Add the target column to each egg
egg_1['Airpointer_NO2[ppb]'] = airpointer['NO2']
egg_2['Airpointer_NO2[ppb]'] = airpointer['NO2']
egg_4['Airpointer_NO2[ppb]'] = airpointer['NO2']

sensor_cluster = pd.concat((egg_1, egg_2, egg_4))
sensor_cluster = sensor_cluster.groupby(sensor_cluster.index).mean()
sensor_cluster.to_pickle('CLEANED_RAW_DATA_HOURLY_WITH_TARGET/sensor_cluster_25May2020_hourly.pkl')